# Importing libraries and dataset from Database

In [1]:
from db import fetch_dataframe, get_db_connection

In [2]:
sql_query = "SELECT * FROM test.data;" # test is the database's name and data is the table
df = fetch_dataframe(sql_query)

Query executed successfully


c:\Users\Legion\Desktop\12-Factor-App\backend\db.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, mydb)


# Exploring the data

In [3]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB


In [5]:
df.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

# Separating the data and target features

In [6]:
X = df.drop('MEDV', axis=1)
Y = df['MEDV']

# Splitting the data into test/train set

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Creating a Pipeline to handle the missing values

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor


pipeline = Pipeline(steps=[
    # ('imputer', SimpleImputer(strategy="median")),
    ('imputer', SimpleImputer(strategy="constant")),
    ('scaler', StandardScaler()),
    ('model', DecisionTreeRegressor())
])

# Fitting the data into model

In [9]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer(strategy='constant')),
                ('scaler', StandardScaler()),
                ('model', DecisionTreeRegressor())])

# Evaluating the model on test set

In [10]:
y_test_pred = pipeline.predict(X_test)

In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error

mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = root_mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

print(mse_test, rmse_test, r2_test, mae_test)

11.74637254901961 3.4272981412505694 0.8398231750387187 2.5656862745098046


# Cross Validation

In [12]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(pipeline, X_train, y_train, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-scores)
rmse_scores.std()

np.float64(0.8741225580843587)

# Saving the model to the database

In [13]:
import joblib
import io

bytes_io = io.BytesIO()
joblib.dump(pipeline, bytes_io)
pipeline_bytes = bytes_io.getvalue()

In [14]:
from model_storage import create_storage_table_pipeline

create_storage_table_pipeline()

In [15]:
sql = get_db_connection()

cursor = sql.cursor()

query = "INSERT INTO model_store_pipeline (model_pipeline, model_blob) VALUES (%s, %s)"
cursor.execute(query, ("model_pipeline", pipeline_bytes))
sql.commit()

cursor.close()
sql.close()